In [9]:
from mastodon import Mastodon

API = Mastodon(
    client_id='b-1LjDLQvdfa1Kd_uVk4EStyVGWCe_lqipdwKG68LGw',
    client_secret='jkWM5f58nofXVQP9-x0sf6yzuB3WfePTiIaHLe97kwM',
    access_token='zuG68yri_Fp4RxFro2QzRK4RRAp1bcmFwyRc5Rjb9Sw',
    api_base_url='https://mastodon.social'  
)

me = API.account_verify_credentials()
print(me)

Account([('id', 115469357557448283), ('username', 'vuTester'), ('acct', 'vuTester'), ('display_name', 'bendover'), ('discoverable', True), ('group', False), ('locked', False), ('created_at', datetime.datetime(2025, 10, 31, 0, 0, tzinfo=tzutc())), ('following_count', 0), ('followers_count', 0), ('statuses_count', 0), ('note', '<p>bendover</p>'), ('url', 'https://mastodon.social/@vuTester'), ('uri', 'https://mastodon.social/ap/users/115469357557448283'), ('avatar', 'https://files.mastodon.social/accounts/avatars/115/469/357/557/448/283/original/a9e38e3d0d41d484.png'), ('header', 'https://files.mastodon.social/accounts/headers/115/469/357/557/448/283/original/f859c2745152ea94.png'), ('avatar_static', 'https://files.mastodon.social/accounts/avatars/115/469/357/557/448/283/original/a9e38e3d0d41d484.png'), ('header_static', 'https://files.mastodon.social/accounts/headers/115/469/357/557/448/283/original/f859c2745152ea94.png'), ('moved', None), ('suspended', None), ('limited', None), ('bot', 

In [ ]:
import time
from datetime import datetime, timedelta, timezone

events = [
    {
        "name": "ChatGPT launch",
        "terms": ["chatgpt", "gpt-4"],
        "event_date": datetime(2022, 11, 30, tzinfo=timezone.utc),
    },
    {
        "name": "Stable Diffusion release",
        "terms": ["stablediffusion", "midjourney"],
        "event_date": datetime(2022, 8, 22, tzinfo=timezone.utc),
    },
]

def to_datetime(dt):
    if isinstance(dt, datetime):
        return dt
    if dt.endswith('Z'):
        dt = dt[:-1] + '+00:00'
    return datetime.fromisoformat(dt)

def count_statuses_for_term_in_window(term, start_dt, end_dt, sleep_sec=1.0, max_loops=50):
    total = 0
    max_id = None
    loops = 0

    while True:
        res = API.search(q=term, result_type="statuses", max_id=max_id)
        statuses = res.get("statuses", [])
        if not statuses:
            break

        created_times = []
        for s in statuses:
            created_at = to_datetime(s["created_at"])
            created_times.append(created_at)
            if start_dt <= created_at < end_dt:
                total += 1

        max_id = min(int(s["id"]) for s in statuses)
        if min(created_times) < start_dt:
            break

        loops += 1
        if loops >= max_loops:
            break

        time.sleep(sleep_sec)

    return total

results = []

for event in events:
    name = event["name"]
    terms = event["terms"]
    event_date = event["event_date"]

    before_start, before_end = event_date - timedelta(days=7), event_date
    after_start, after_end = event_date, event_date + timedelta(days=7)

    print(f"\n=== Event: {name} ===")
    event_result = {"name": name, "before": {}, "after": {}}

    for term in terms:
        print(f"\nTerm: '{term}'")
        before_count = count_statuses_for_term_in_window(term, before_start, before_end)
        print(f"  Before: {before_count}")
        after_count = count_statuses_for_term_in_window(term, after_start, after_end)
        print(f"  After:  {after_count}")

        event_result["before"][term] = before_count
        event_result["after"][term] = after_count

    results.append(event_result)


=== Event: ChatGPT launch ===

Term: 'chatgpt'
  Before: 0
